# Basic Example

In [1]:
import csbsingleenv

In [2]:
env = csbsingleenv.CodersStrikeBackSingle()

In [3]:
env.reset()
done = False
rewards = []
while not done:
    obs, reward, done, _, _ = env.step(env.action_space.sample())
    rewards.append(reward)
print(sum(rewards))

-10100


In [4]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

2024-05-07 17:50:55,039	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [5]:
if ray.is_initialized():
  ray.shutdown()
ray.init(num_cpus=4)

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-05-07 15:52:56,397	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.12
Ray version:,2.7.0
Dashboard:,http://127.0.0.1:8265


In [6]:
config = (PPOConfig()
          .environment(csbsingleenv.CodersStrikeBackSingle)
          .framework('torch')
          .training(gamma=0.9)
          .rollouts(num_rollout_workers=3)
)

stop = {"timesteps_total": 12000}

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop),
)

tuner.fit()

2024-05-07 15:52:59,157	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=1819652) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=1819652) 2024-05-07 15:53:08,844	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_con

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'__all__': {'num_agent_steps_trained': 128.0, 'num_env_steps_trained': 4000.0, 'total_loss': 7.225998489078936}, 'default_policy': {'total_loss': 7.225998489078936, 'policy_loss': -0.05337565690815163, 'vf_loss': 7.266583819252087, 'vf_loss_unclipped': 5121846.473598326, 'vf_explained_var': -8.715400055273256e-05, 'entropy': 4.161632999936655, 'mean_kl_loss': 0.028422914042414576, 'default_optimizer_lr': 5.000000000000001e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.675000011920929}}, 'num_env_steps_sampled': 12000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 12000, 'num_agent_steps_trained': 0}, 'sampler_results': {'episode_reward_max': -10100.0, 'episode_reward_min': -10135.0, 'episode_reward_mean': -10101.96, 'episode_len_mean': 101.96, 'episode_media': {}, 'episodes_this_iter': 39, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mea

# Empty Customized Environment

In [7]:
import csbsingleenv

import sys
# Assuming your Python file is in 'src' under 'my_project' directory
sys.path.append('../..')
import env_customizer

In [8]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

In [9]:
if ray.is_initialized():
  ray.shutdown()

runtime_env = {"py_modules": ["../.."]}
ray.init(num_cpus=4, runtime_env=runtime_env)

2024-05-07 15:54:09,005	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2024-05-07 15:54:09,760	INFO packaging.py:518 -- Creating a file package for local directory '../..'.
2024-05-07 15:54:10,328	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_9298b2e6c4d9ae7e.zip' (11.03MiB) to Ray cluster...
2024-05-07 15:54:10,367	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_9298b2e6c4d9ae7e.zip'.


Python version:,3.10.12
Ray version:,2.7.0
Dashboard:,http://127.0.0.1:8265


In [ ]:
config = (PPOConfig()
          .environment(env_customizer.CustomizedEnvironment,
                      env_config={"env" : csbsingleenv.CodersStrikeBackSingle()})
          .framework('torch')
          .training(gamma=0.9)
          .rollouts(num_rollout_workers=3)
)

stop = {"timesteps_total": 12000}

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop),
)

tuner.fit()

# Real Customized Environment

In [18]:
import csbsingleenv

import sys
# Assuming your Python file is in 'src' under 'my_project' directory
sys.path.append('../..')
import env_customizer

In [45]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces, logger

# Note, this is only meant as an example of how to write a custom environment.
# I doubt it'll actually learn well.
class CustomizedRacing:
    def __init__(self):
        min_pos = -200000.0
        max_pos = 200000.0
        min_vel = -2000.0
        max_vel = 2000.0
        screen_max = [16000,9000]
        # This observation space will only have the next checkpoint rather than the next 5.
        self.observation_space = spaces.Box(
            low=np.array([0, -np.pi, min_pos, min_pos, min_vel, min_vel]+[0,0]),
            high=np.array([3, np.pi, max_pos, max_pos, max_vel, max_vel]+screen_max),
            dtype=np.float64
        )

        # This action will be how much to turn, rather than a point to turn towards.
        self.action_space = spaces.Box(
            low = np.array([-1.0, 0.0]),
            high = np.array([1.0, 100]),
            dtype=np.float64
        )

    # We can define and reset any extra variables we want to keep track of here.
    def reset(self):
        self.last_checkpoint = None
        self.last_sqr_dist = None

    def reward(self, env, rew):
        rew = -1
        # Add reward for getting closer to the next checkpoint
        cur_dist = csbsingleenv.get_sqr_distance(env.pos, env.next_checkpoint())
        if self.last_sqr_dist is not None and cur_dist < self.last_sqr_dist:
            rew += 0.5
        self.last_sqr_dist = cur_dist
        
        # Add reward for passing a checkpoint
        cur_checkpoint = env.checkpoint_index
        if self.last_checkpoint is not None and cur_checkpoint != self.last_checkpoint:
            rew += 150
        self.last_checkpoint = cur_checkpoint
        
        if env.completed:
            rew += 500
        if env.failed:
            rew -= 500
        return rew
    
    # Need to define the traformation.  Should be easy here since we just need to drop the excess data
    def transform_observation(self, env, obs):
        return obs[:8]

    # Need to also transform the angle change into a point to aim at.
    # Will define it as 1000 distance from the angle relative to current pointed direction
    def transform_action(self, env, action):
        preferred_dir = env.theta + action[0]*env.maxSteeringAngle
        dir_vector = 1000*csbsingleenv.angle_to_vector(preferred_dir)
        point_to_aim_at = env.pos + dir_vector
        trans_action = np.array([point_to_aim_at.x, point_to_aim_at.y, action[1]])
        return trans_action

In [46]:
env = env_customizer.CustomizedEnvironment(
    env_config={"env":csbsingleenv.CodersStrikeBackSingle(),
                "customizer":CustomizedRacing()})

In [47]:
env.reset()

(array([ 3.00000000e+00, -1.44213571e+00,  1.14580000e+04,  6.77000000e+03,
         0.00000000e+00,  0.00000000e+00,  1.17500000e+04,  4.51300000e+03]),
 {})

In [49]:
env.reset()
done = False
while not done:
    obs, rew, done, _, _ = env.step(np.array([0,100]))
    print(rew)

-1
-0.5
-0.5
-0.5
-0.5
-0.5
149
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-501


In [98]:
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

In [50]:
if ray.is_initialized():
  ray.shutdown()

runtime_env = {"py_modules": ["../.."]}
ray.init(num_cpus=4, runtime_env=runtime_env)

2024-05-08 12:09:45,088	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2024-05-08 12:09:45,581	INFO packaging.py:518 -- Creating a file package for local directory '../..'.
2024-05-08 12:09:45,772	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_1bbf421b9294fd02.zip' (22.79MiB) to Ray cluster...
2024-05-08 12:09:45,820	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_1bbf421b9294fd02.zip'.


Python version:,3.10.12
Ray version:,2.7.0
Dashboard:,http://127.0.0.1:8265


In [100]:
config = (PPOConfig()
          .environment(env_customizer.CustomizedEnvironment,
                      env_config={"env" : csbsingleenv.CodersStrikeBackSingle(),
                                 "customizer" : CustomizedRacing()})
          .framework('torch')
          .training(gamma=0.99)
          .rollouts(num_rollout_workers=3)
)

stop = {"timesteps_total": 100000}

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop),
)

tuner.fit()

2024-04-28 22:02:10,932	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=2440749) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=2440749) 2024-04-28 22:02:14,823	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_con

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'__all__': {'num_agent_steps_trained': 128.0, 'num_env_steps_trained': 4000.0, 'total_loss': 8.874769193022999}, 'default_policy': {'total_loss': 8.874769193022999, 'policy_loss': -0.028919879482117796, 'vf_loss': 8.889652047838483, 'vf_loss_unclipped': 427.2954352730627, 'vf_explained_var': -0.0033682089116273406, 'entropy': 2.986706386624115, 'mean_kl_loss': 0.013863695907958147, 'default_optimizer_lr': 5.000000000000001e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 1.0125000476837158}}, 'num_env_steps_sampled': 100000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 100000, 'num_agent_steps_trained': 0}, 'sampler_results': {'episode_reward_max': 172.5, 'episode_reward_min': -94.0, 'episode_reward_mean': -57.38, 'episode_len_mean': 106.63, 'episode_media': {}, 'episodes_this_iter': 41, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {

# Create Dataset for Behavioral Cloning

In [45]:
import csbsingleenv
import numpy as np
env = csbsingleenv.CodersStrikeBackSingle()
# Set pseudorandom seed for the getting the same game
env.seed(1234)
state_actions = []
for n in range(100):
    state, _ = env.reset()
    done = False
    while not done:
        # Take action (simple policy)
        targetX, targetY = state[6:8]
        thrust = 100
        action = np.array([targetX, targetY, thrust], dtype=np.float32)
        state_actions.append([state, action])
        # Do a game step
        state, reward, done, trunc, _ = env.step(action)
        
env.close()

In [46]:
import pandas as pd
df = pd.DataFrame(state_actions, columns=['obs','actions'])

In [52]:
import ray.data as rd
ray_dataset = rd.from_pandas(df)

In [56]:
ray_dataset.write_json(".")

2024-05-08 12:12:30,233	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[Write]
2024-05-08 12:12:30,234	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-05-08 12:12:30,235	INFO streaming_executor.py:96 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

In [58]:
tt = ray.data.read_json("racing_pretrain.json")

2024-05-08 12:13:08,153	INFO read_api.py:406 -- To satisfy the requested parallelism of 8, each read task output is split into 8 smaller blocks.


In [59]:
tt.take_batch(3)

2024-05-08 12:13:11,603	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadJSON->SplitBlocks(8)] -> LimitOperator[limit=3]
2024-05-08 12:13:11,605	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-05-08 12:13:11,607	INFO streaming_executor.py:96 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

{'obs': array([array([ 3.00000000e+00, -1.97959814e+00,  7.22300000e+03,  6.40100000e+03,
                0.00000000e+00,  0.00000000e+00,  4.70000000e+03,  5.77000000e+02,
                1.86900000e+03,  3.22100000e+03,  3.08900000e+03,  5.10000000e+03,
                7.22300000e+03,  6.40100000e+03,  4.70000000e+03,  5.77000000e+02]),
        array([ 3.00000000e+00, -1.97959814e+00,  7.18300000e+03,  6.30900000e+03,
               -3.31500000e+01, -7.73500000e+01,  4.70000000e+03,  5.77000000e+02,
                1.86900000e+03,  3.22100000e+03,  3.08900000e+03,  5.10000000e+03,
                7.22300000e+03,  6.40100000e+03,  4.70000000e+03,  5.77000000e+02]),
        array([ 3.00000000e+00, -1.97957685e+00,  7.11000000e+03,  6.14000000e+03,
               -6.12000000e+01, -1.43650000e+02,  4.70000000e+03,  5.77000000e+02,
                1.86900000e+03,  3.22100000e+03,  3.08900000e+03,  5.10000000e+03,
                7.22300000e+03,  6.40100000e+03,  4.70000000e+03,  5.7700000